In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import openpyxl

In [2]:
df = pd.read_csv('./Data/train.csv')

C:\anaconda\envs\Hack\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
def floor_recognition(s):
    if s[0] == "-":
        return -1 * floor_recognition(s[1:])
    if s.isdigit():
        return int(s)
    elif "подв" in s:
        return -1
    elif "цок" in s:
        return 0
    else:
        return -100
    
def floor_parser(row):
    #"<=0",  "2-5", "6-10", "11-20", "20<"
    result = [0, 0, 0, 0, 0]
    r = str(row).lower()
    for stop_word in [".0", "этажа", "этаж", "эт", " ", "-й"]:
        r = r.replace(stop_word, "")
    r = r.replace(".", ",").split(",")[0]
    floor = floor_recognition(r)
    if floor < -99:
        return pd.Series(result)
    elif floor < 1:
        result[0] = 1
    elif floor <=5:
        result[1] = 1
    elif floor <= 10:
        result[2] = 1
    elif floor <= 20:
        result[3] = 1
    else:
        result[4] = 1
    return pd.Series(result)

In [8]:
df[["floor<0", "floor<=5", "floor<=10", "floor<=20", "floor>20"]] = df["floor"].apply(floor_parser)

In [26]:
df[df["region"] == "Коми"]["osm_subway_closest_dist"].tail(10)

278870    657.836399
278871    650.894278
278901    900.129843
279012    694.167865
279118    652.694605
279249    665.783440
279409    657.493021
279535    654.756046
279663    668.110228
279762    900.270000
Name: osm_subway_closest_dist, dtype: float64

In [16]:
russian_states = pd.read_excel('Data/Russian_state.xlsx')

In [17]:
def a(row):
    aa = str(row)
    bb = russian_states[ russian_states["region"] == aa] ["state"]
    if len(bb) > 0:
        return bb.item()
    else:
        print ([aa])
df["state"] = df["region"].apply(a)

In [30]:
X = df[["lat", "lng", "floor<0", "floor<=5", "floor<=10", "floor<=20", "floor>20", "realty_type", "total_square", "state", "price_type", "osm_amenity_points_in_0.005", "osm_building_points_in_0.005", "osm_catering_points_in_0.005", "osm_city_closest_dist", "osm_city_nearest_population", "osm_crossing_points_in_0.005", "osm_culture_points_in_0.005", "osm_finance_points_in_0.005", "osm_healthcare_points_in_0.005", "osm_historic_points_in_0.005", "osm_hotels_points_in_0.005", "osm_leisure_points_in_0.005", "osm_offices_points_in_0.005", "osm_shops_points_in_0.005", "osm_subway_closest_dist", "osm_train_stop_closest_dist", "osm_transport_stop_closest_dist", "osm_transport_stop_points_in_0.005", "reform_count_of_houses_500", "reform_house_population_500", "reform_mean_floor_count_500", "reform_mean_year_building_500"]] 

In [37]:
X["reform_house_population_500"].fillna( X["reform_house_population_500"].mean(), inplace = True)
X["reform_mean_floor_count_500"].fillna( X["reform_mean_floor_count_500"].mean(), inplace = True)
X["reform_mean_year_building_500"].fillna( X["reform_mean_year_building_500"].mean(), inplace = True)

In [56]:
X["target"] = df["per_square_meter_price"]

In [57]:
X.head()

,lat,lng,floor<0,floor<=5,floor<=10,floor<=20,floor>20,realty_type,total_square,price_type,...,state_Московская область,state_Приволжский,state_Санкт-Петербург,state_Северо-Западный,state_Северо-Кавказский,state_Сибирский,state_Уральский,state_Центральный,state_Южный,target
0,57.998207,56.292797,0,0,0,0,0,10,32.0,0,...,0,1,0,0,0,0,0,0,0,139937.500000
1,55.574284,39.543835,0,0,0,0,0,10,280.0,0,...,1,0,0,0,0,0,0,0,0,60410.714286
2,57.619140,39.850525,0,0,0,0,0,110,297.4,0,...,0,0,0,0,0,0,0,1,0,45164.761264
3,53.897083,87.108604,0,0,0,0,0,110,190.0,0,...,0,0,0,0,0,1,0,0,0,28805.263158
4,55.802590,37.487110,0,0,0,0,0,10,60.2,0,...,0,0,0,0,0,0,0,0,0,13222.591362


In [47]:
X = pd.get_dummies(X, drop_first=True) 

In [60]:
X.to_csv('./Data/train_parse1.csv', index = False)